In [1]:
!pip install reformer_pytorch
!git clone --recursive https://github.com/hjk1996/pm_prediction.git
%cd pm_prediction

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2901 sha256=27991cebd76161f0173c60786925869c7e6a2bddcda03e2d232c978d7421e72f
  Stored in directory: /root/.cache/pip/wheels/b1/cb/39/7ce7ff2d2fd37cfe1fe7b3a3c43cf410632b2ad3b3f3986d73
  Created wheel for product-key-memory: filename=product_key_memory-0.1.10-py3-none-any.whl size=3068 sha256=8bb65596419bbefe792ccb17b4174f7b154886ebf9abb1ebea8e382dca990b95
  Stored in directory: /root/.cache/pip/wheels/2c/97/82/f94ef75772952e1eaec19864fc840620ec94d9ac7c9c0b6823
Successfully built axial-positional-embedding product-key-memory
Cloning into 'pm_prediction'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 

In [2]:
import sys
import os
sys.path += [os.path.abspath("./Autoformer")]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import random
import argparse
import torch
import argparse
from Autoformer.data_provider.data_loader import CustomPM
from Autoformer.exp.exp_main import Exp_Main
import datetime
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def get_pm_files(path: str, interpolate=True):
    paths = glob(os.path.join(path, '*.csv'))
    paths.sort()
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        if interpolate:
            df.interpolate(inplace=True)
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

def get_aws_files(path :str, interpolate=True):
    paths = glob(os.path.join(path, '*.csv'))
    paths.sort()
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        if interpolate:
            df.interpolate(inplace=True)
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

def get_aws_map(path: str) -> dict:
    df = pd.read_csv(path)
    aws_map = {}
    for row in df.itertuples():
        aws_map[row[1]] = (row[2], row[3])
    
    return aws_map

def get_pm_map(path: str) -> dict:
    df = pd.read_csv(path)
    pm_map = {}
    for row in df.itertuples():
        pm_map[row[1]] = (row[2], row[3])
    
    return pm_map


In [6]:


def find_n_nearest_points(point_name: str, n: int, aws_map: dict, pm_map:dict) -> list:
    point = pm_map[point_name]
    distances = []
    for key, value in aws_map.items():
        distances.append((key, np.linalg.norm(np.array(point) - np.array(value))))
    
    distances.sort(key=lambda x: x[1])
    return distances[:n]

In [7]:
# 위도 경도 스케일링을 위한 값 (min-max)
lat_max = 37.0106
lat_min = 36.0625
lon_max = 127.4938
lon_min = 125.5595


In [8]:
def merge_n_nearest(aws: pd.DataFrame, pm: pd.DataFrame, n: int, aws_map: dict, pm_map: dict):
    selected_columns = ['연도', '일시', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)', 'lat', 'lon']
    year_map = {0: '2017', 1: '2018', 2: '2019', 3: '2020', 4: '2021'}
    pms = []
    aws['lat'] = aws['lat'].map(lambda x: (x - lat_min) / (lat_max - lat_min))
    aws['lon'] = aws['lon'].map(lambda x: (x - lon_min) / (lon_max - lon_min))
    pm['lat'] = pm['lat'].map(lambda x: (x - lat_min) / (lat_max - lat_min))
    pm['lon'] = pm['lon'].map(lambda x: (x - lon_min) / (lon_max - lon_min))
    for key, _ in pm_map.items():
        distances = find_n_nearest_points(key, n, aws_map, pm_map)
        sub_pm =  pm[pm['측정소'] == key]
        for i, dist in enumerate(distances):
            dist_name = dist[0]
            sub_aws = aws[aws['지점'] == dist_name]
            sub_aws = sub_aws[selected_columns]
            sub_pm =  pd.merge(sub_aws, sub_pm, on=['연도', '일시'], how='right', suffixes=(f'_aws{i}', f'_pm{i}'))
        
        sub_pm['date'] = sub_pm['연도'].map(lambda x: year_map[x]) + "-" + sub_pm['일시']
        sub_pm['date'] = pd.to_datetime(sub_pm['date'], format='%Y-%m-%d %H:%M')
        sub_pm.drop(['연도', '일시'], axis=1, inplace=True)
        sub_pm.insert(0, 'date', sub_pm.pop('date'))
        sub_pm.bfill(inplace=True)
        sub_pm.drop(['측정소'], axis=1, inplace=True)


        
        pms.append(sub_pm)

    
    return pms


In [9]:
def add_lat_long_to_pm(df: pd.DataFrame, pm_map: dict):
    df['lat'] = df['측정소'].map(lambda x: pm_map[x][0])
    df['lon'] = df['측정소'].map(lambda x: pm_map[x][1])
    return df


def add_lat_long_to_aws(df: pd.DataFrame, aws_map: dict):
    df['lat'] = df['지점'].map(lambda x: aws_map[x][0])
    df['lon'] = df['지점'].map(lambda x: aws_map[x][1])
    return df

In [16]:
def make_train_data(aws: pd.DataFrame, pm: pd.DataFrame, pm_map: dict, aws_map: dict, k :int= 3,  save_path: str = './dataset/pm_for_train'):
    aws = add_lat_long_to_aws(aws, aws_map)
    pm = add_lat_long_to_pm(pm, pm_map)
    data = merge_n_nearest(aws, pm, k, aws_map, pm_map)
    if save_path:
      os.makedirs(save_path, exist_ok=True)
    for i, df in enumerate(data):
        df.to_csv(os.path.join(save_path, f'pm_{i}.csv'), index=False)
    return data


In [17]:
def make_test_data(aws: pd.DataFrame, pm: pd.DataFrame, pm_map: dict, aws_map: dict, k: int = 3):
    chunks = []
    aws.dropna(inplace=True)
    pm.dropna(inplace=True)
    aws = add_lat_long_to_aws(aws, aws_map)
    pm = add_lat_long_to_pm(pm, pm_map)
    data = merge_n_nearest(aws, pm, k, aws_map, pm_map)
    data = [df.iloc[:-48] for df in data]
    data = pd.concat(data, ignore_index=True)
    cols = list(data.columns)
    cols.remove('PM2.5')
    cols.remove('date')
    data = data[["date"] + cols + ['PM2.5']]

    x_data = data[data.columns[1:]]
    print(len(data))
    x_data = np.array(x_data)
  


  

    for i in range(0, len(x_data), 48):
        x = x_data[i:i+48]
        stamp = data['date'][i:i+48]
        stamp = pd.DataFrame(stamp, columns=['date'])
        last_date = stamp['date'].iloc[-1]
        test_date = [last_date + datetime.timedelta(hours=x) for x in range(1, 73)]
        test_stamp = pd.DataFrame(test_date, columns=['date'])
        stamp = pd.concat([stamp, test_stamp], ignore_index=True)
        stamp['month'] = stamp.date.apply(lambda row: row.month, 1)
        stamp['day'] = stamp.date.apply(lambda row: row.day, 1)
        stamp['weekday'] = stamp.date.apply(lambda row: row.weekday(), 1)
        stamp['hour'] = stamp.date.apply(lambda row: row.hour, 1)
        stamp.drop(['date'], axis=1, inplace=True)
        x_mark = stamp.values[:48]
        y_mark = stamp.values[48-24:]
        dec_inp = torch.zeros(48, x.shape[-1])
        dec_inp  = torch.cat([torch.tensor(x[24:, :]), dec_inp], dim=0)



        chunks.append((x, x_mark, dec_inp, y_mark))



    return chunks

In [18]:
# 대회 데이터 구글 드라이브 pollution/dataset 폴더 안에 배치해야함
aws_map = get_aws_map('/content/drive/MyDrive/pollution/dataset/META/awsmap.csv')
pm_map = get_pm_map('/content/drive/MyDrive/pollution/dataset/META/pmmap.csv')
train_pm = get_pm_files('/content/drive/MyDrive/pollution/dataset/TRAIN', interpolate=True)
train_aws = get_aws_files('/content/drive/MyDrive/pollution/dataset/TRAIN_AWS', interpolate=True)
test_pm = get_pm_files('/content/drive/MyDrive/pollution/dataset/TEST_INPUT', interpolate=False)
test_aws = get_aws_files('/content/drive/MyDrive/pollution/dataset/TEST_AWS', interpolate=False)

In [19]:
# 훈련에 사용될 train dataframe은 pm_prediction/dataset/pm_for_train 폴더 안에 저장됨
train_data = make_train_data(train_aws, train_pm, pm_map, aws_map)
test_data = make_test_data(test_aws, test_pm, pm_map, aws_map)

52224


In [20]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = len(data)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

In [21]:
# 리더보드 제출용 정답을 만들기 위한 테스트 데이터
test_dataset = TestDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [22]:
# 입력 차원 확인용 데이터셋
dataset = CustomPM(data_path='./dataset/pm_for_train')

In [23]:
class DotDict(dict):
    def __getattr__(self, name):
        return self[name]

    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [24]:
# 모델 및 훈련 설정

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

args = DotDict({
    #basic
    'is_training': True,
    'model_id': 'test',
    'model': 'Autoformer',
    'data': 'customPM',
    'root_path': './dataset',
    'data_path': 'pm_for_train',
    'features': 'MS',
    'target': 'PM2.5',
    'freq': 'h',
    'checkpoints': './checkpoints',
    #forecasting task
    'seq_len': 48, # seq_len + label_len = pred_len 이여야함.
    'label_len': 24,
    'pred_len': 72,
    #model config
    'bucket_size':4,
    'n_hashes': 4,
    'enc_in': dataset[0][0].shape[1],
    'dec_in': dataset[0][0].shape[1],
    'c_out': 1,
    'd_model': 512,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 2,
    'd_ff': 2048,
    'moving_avg': 25,
    'factor': 1,
    'distil': True,
    'dropout': 0.05,
    'embed': 'timeF',
    'activation': 'gelu',
    'output_attention': False,
    #optimization
    'num_workers': 8,
    'itr': 1,
    'train_epochs': 2, # train_epoch 수
    'batch_size': 32,
    'patience': 3,
    'learning_rate': 0.0001,
    'des': 'test',
    'loss': 'mse',
    'lradj': 'type1',
    'use_amp': False,

    #GPU
    'use_gpu': False,
    'gpu': 0,
    'use_multi_gpu': False,
    'devices': '0,1,2,3',  
})

In [25]:
#모델

Exp = Exp_Main
exp = Exp(args)  

setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 1)

Use CPU


In [ ]:
# 모델 훈련
exp.train(setting)
torch.cuda.empty_cache()

torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])
torch.Size([32, 96, 24])


In [ ]:
# 훈련된 모델로 테스트 데이터 예측. 결과 데이터프레임으로 저장됨. pm_pollution/results 폴더 안에서 찾을 수 있음
exp.predict(setting, test_loader, load=False )

In [ ]:
# indepent variables

# year (int): start from 0
# datetime (str): 'MM-DD HH:MM' format
# point (str): categoricla value, 30 unique values
# temp (float): temperature in celsius. range from 0 to 1
# degree (float): wind direction in degree. range from 0 to 1
# speed (float): wind speed in m/s. range from 0 to 1
# rain (float): rain in mm. range from 0 to 1
# humidity (float): relative humidity in %. range from 0 to 1
# station (str): categorical value, 17 unique values, place where the data was collected

# dependent variables

# PM2.5 (float): PM2.5 concentration in ug/m3. range from 0 to 1

In [ ]:
# 17 list
# 24544 items in each list
# total length = 
len(dataset.xs[0])

NameError: name 'dataset' is not defined